In [1]:
library(survival)
library(foreach)
library(doParallel)
library(ggplot2)
library(rem)

Loading required package: iterators
Loading required package: parallel


In [2]:
packageVersion("rem")

[1] ‘1.2.8’

In [3]:
packageVersion("survival")

[1] ‘2.41.3’

## Read the data

In [5]:
# Orbis dataset, 2015 snapshot
# Danish firms with operating revenue > 10k US dollars
# Appointments to 'board of directors' during 1994-2014

dt <- read.table("/data.csv", sep = ",", na.strings = "NA", header = TRUE, dec = ".", stringsAsFactors = FALSE)
head(dt)

X,companyBvDID,companyName,companyCountry,companyCity,companyMajorSector,companyNACESector,companyEmployees,companyOperatingRevenue,companyAssets,⋯,positionCurrent,positionType.1,positionAppointment,positionResignation,appDate,year,giant1,giant2,currentDate,duration
1285,DK41853816,HALDOR TOPSOE A/S,DK,KGS.LYNGBY,Other services,71,2694,928709,1054497,⋯,1,Board of Directors,1994-09-14,NA,1994-09-14,1994,True,True,2015-12-01,7748 days 00:00:00.000000000
1577,DK17106589,LAERERNES PENSION FORSIKRINGSAKTIESELSKAB,DK,HELLERUP,Insurance companies,65,NA,732330,13415875,⋯,1,Board of Directors,1994-12-10,NA,1994-12-10,1994,True,True,2015-12-01,7661 days 00:00:00.000000000
1837,DK44782510,AVK HOLDING A/S,DK,GALTEN,Other services,70,3153,587165,623792,⋯,1,Board of Directors,1994-12-14,NA,1994-12-14,1994,True,True,2015-12-01,7657 days 00:00:00.000000000
2341,DK11369677,POLAR SEAFOOD DENMARK A/S,DK,VODSKOV,Wholesale & retail trade,46,563,404514,227416,⋯,1,Board of Directors,1994-09-28,NA,1994-09-28,1994,True,True,2015-12-01,7734 days 00:00:00.000000000
2352,DKGL67900,POLAR SEAFOOD GREENLAND A/S,DK,NUUK (GODTHAEB),Wholesale & retail trade,46,835,402950,336934,⋯,1,Board of Directors,1994-09-28,NA,1994-09-28,1994,True,True,2015-12-01,7734 days 00:00:00.000000000
2353,DKGL67900,POLAR SEAFOOD GREENLAND A/S,DK,NUUK (GODTHAEB),Wholesale & retail trade,46,835,402950,336934,⋯,1,Board of Directors,1994-09-28,NA,1994-09-28,1994,True,True,2015-12-01,7734 days 00:00:00.000000000


In [6]:
nrow(dt)

[1] 7582

## Data preparation

In [ ]:
# In order to estimate relational event models, the events have to be ordered,
# either according to an ordinal or a continuous event sequence.

# The ordinal event sequence simply orders the events and gives each event a place in the sequence.

# The continuous event sequence creates an artificial sequence,
# ranging from min(datevar) to max(datevar) and matches each event with its place in the artificial event sequence

# sort by appDate
dt[order(dt$appDate),]

In [8]:
# Create event sequence and order the data
dt <- eventSequence(datevar = dt$appDate, dateformat = '%Y-%m-%d',
                    data = dt, type = 'continuous',
                    byTime = 'yearly', returnData = TRUE,
                    sortData = TRUE)

In [ ]:
# Create counting process data set (with null-events): conditional logit setting
dts <- createRemDataset(data = dt, sender = dt$companyBvDID, target = dt$positionUCI,
                          eventSequence = dt$event.seq.cont, atEventTimesOnly = TRUE, returnInputData = TRUE)

In [8]:
## Divide up the results: counting process data = 1, original data = 2
dtrem <- dts[[1]]
dt <- dts[[2]]

In [9]:
# Sort the data set
dtrem <- dtrem[order(dtrem$eventTime), ]

In [10]:
## Merge all necessary event attribute variables back in
dtrem$companyBvDID <- dt$companyBvDID[match(dtrem$eventID, dt$eventID)]
dtrem$companyName <- dt$companyName[match(dtrem$eventID, dt$eventID)]
dtrem$companyCity <- dt$companyCity[match(dtrem$eventID, dt$eventID)]
dtrem$companyMajorSector <- dt$companyMajorSector[match(dtrem$eventID, dt$eventID)]
dtrem$companyNACESector <- dt$companyNACESector[match(dtrem$eventID, dt$eventID)]
dtrem$companyEmployees <- dt$companyEmployees[match(dtrem$eventID, dt$eventID)]
dtrem$companyOperatingRevenue <- dt$companyOperatingRevenue[match(dtrem$eventID, dt$eventID)]
dtrem$companyAssets <- dt$companyAssets[match(dtrem$eventID, dt$eventID)]
dtrem$companyMarketCap <- dt$companyMarketCap[match(dtrem$eventID, dt$eventID)]
dtrem$companyTypeEntity <- dt$companyTypeEntity[match(dtrem$eventID, dt$eventID)]
dtrem$companyTickerSymbol <- dt$companyTickerSymbol[match(dtrem$eventID, dt$eventID)]
dtrem$companyStatus <- dt$companyStatus[match(dtrem$eventID, dt$eventID)]
dtrem$companyMainExchange <- dt$companyMainExchange[match(dtrem$eventID, dt$eventID)]
dtrem$companyGUO <- dt$companyGUO[match(dtrem$eventID, dt$eventID)]
dtrem$positionUCI <- dt$positionUCI[match(dtrem$eventID, dt$eventID)]
dtrem$positionLastName <- dt$positionLastName[match(dtrem$eventID, dt$eventID)]
dtrem$positionFullName <- dt$positionFullName[match(dtrem$eventID, dt$eventID)]
dtrem$positionGender <- dt$positionGender[match(dtrem$eventID, dt$eventID)]
dtrem$positionNationality <- dt$positionNationality[match(dtrem$eventID, dt$eventID)]
dtrem$positionResidence <- dt$positionResidence[match(dtrem$eventID, dt$eventID)]
dtrem$positionType <- dt$positionType[match(dtrem$eventID, dt$eventID)]
dtrem$positionResponsability <- dt$positionResponsability[match(dtrem$eventID, dt$eventID)]
dtrem$appDate <- dt$appDate[match(dtrem$eventID, dt$eventID)]
dtrem$year <- dt$year[match(dtrem$eventID, dt$eventID)]
dtrem$duration <- dt$duration[match(dtrem$eventID, dt$eventID)]

In [11]:
table(dtrem$eventDummy)
# We have 123,347 events in total (both observed and potential)
# 115,765 of events are potential, 7,582 are observed


     0      1 
115765   7582 

In [12]:
summary(dtrem$eventTime)
# Time: yearly, there are 20 years

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   5.000   9.000   9.313  14.000  21.000 

## Calculating network statistics

In [14]:
### Calculate network covariates with the halflife = 2 years

# Popularity
dtrem$dirPopularity2 <- degreeStat(data=dtrem, time=dtrem$eventTime,
                                  degreevar=dtrem$target, halflife=2,
                                eventvar = dtrem$eventDummy, 
                                   showprogressbar = FALSE, 
                                   returnData = FALSE)

# Four-cycle
dtrem$fourCycle2 <- fourCycleStat(data=dtrem, time=dtrem$eventTime,
                                  sender=dtrem$sender,target=dtrem$target, 
                                  halflife=2, eventvar = dtrem$eventDummy,
                                  showprogressbar = FALSE, 
                                   returnData = FALSE)

# Calculate z-scores of the network variables
dtrem$dirPopularity2_z <- scale(dtrem$dirPopularity2, center = TRUE, scale = TRUE)
dtrem$fourCycle2_z <- scale(dtrem$fourCycle2, center = TRUE, scale = TRUE)

## Models

In [61]:
# Recode gender 
dtrem$gender_man <- dtrem$positionGender
dtrem$gender_man[dtrem$positionGender == '1'] <- 0
dtrem$gender_man[dtrem$positionGender == '0'] <- 1

# Recode financial firms and banks
dtrem$entity <- dtrem$companyTypeEntity
dtrem$entity <- factor(dtrem$entity)
dtrem$entity_finb[dtrem$entity == 'Bank'] <- 1
dtrem$entity_finb[dtrem$entity == 'Financial company'] <- 1
dtrem$entity_finb[dtrem$entity == 'Foundation/Research institute'] <- 0
dtrem$entity_finb[dtrem$entity == 'Industrial company'] <- 0
dtrem$entity_finb[dtrem$entity == 'Insurance company'] <- 0
dtrem$entity_finb[dtrem$entity == 'Mutual and pension fund/Nominee/Trust/Trustee'] <- 0
dtrem$entity_finb[dtrem$entity == 'Private equity firm'] <- 0

In [62]:
df2 <- dtrem[order(dtrem$eventTime), ]
is.sorted = Negate(is.unsorted)
is.sorted(df2$eventTime)

[1] TRUE

In [63]:
# Delete first 4 years = halflife*2
dtrem2 <- subset(df2, df2$eventTime > 4)
nrow(df2);nrow(dtrem2)

[1] 123347

[1] 93325

In [64]:
# Run models

m01 <- coxph(Surv(rep(1, nrow(dtrem2)), eventDummy) ~ dirPopularity2_z + strata(eventTime),
             data=dtrem2)

m02 <- coxph(Surv(rep(1, nrow(dtrem2)), eventDummy) ~ dirPopularity2_z + I(dirPopularity2_z^2) + 
             strata(eventTime), data=dtrem2)

m03 <- coxph(Surv(rep(1, nrow(dtrem2)), eventDummy) ~ fourCycle2_z + strata(eventTime), data=dtrem2)

m04 <- coxph(Surv(rep(1, nrow(dtrem2)), eventDummy) ~ dirPopularity2_z + I(dirPopularity2_z^2) +
             fourCycle2_z + strata(eventTime), data=dtrem2)

m05 <- coxph(Surv(rep(1, nrow(dtrem2)), eventDummy) ~ dirPopularity2_z + I(dirPopularity2_z^2) +
                fourCycle2_z + gender_man + nationality_dk + entity_finb + companyOperatingRevenue
                + strata(eventTime), data=dtrem2)

In [65]:
summary(m01)

Call:
coxph(formula = Surv(rep(1, nrow(dtrem2)), eventDummy) ~ dirPopularity2_z + 
    strata(eventTime), data = dtrem2)

  n= 93325, number of events= 7374 

                     coef exp(coef) se(coef)     z Pr(>|z|)    
dirPopularity2_z 0.063304  1.065351 0.005629 11.24   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

                 exp(coef) exp(-coef) lower .95 upper .95
dirPopularity2_z     1.065     0.9387     1.054     1.077

Concordance= 0.557  (se = 0.01 )
Rsquare= 0.001   (max possible= 0.713 )
Likelihood ratio test= 108.3  on 1 df,   p=0
Wald test            = 126.5  on 1 df,   p=0
Score (logrank) test = 127.8  on 1 df,   p=0


In [66]:
summary(m02)

Call:
coxph(formula = Surv(rep(1, nrow(dtrem2)), eventDummy) ~ dirPopularity2_z + 
    I(dirPopularity2_z^2) + strata(eventTime), data = dtrem2)

  n= 93325, number of events= 7374 

                           coef exp(coef)  se(coef)      z Pr(>|z|)    
dirPopularity2_z       0.157375  1.170435  0.013214 11.909  < 2e-16 ***
I(dirPopularity2_z^2) -0.011312  0.988751  0.001562 -7.241 4.45e-13 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

                      exp(coef) exp(-coef) lower .95 upper .95
dirPopularity2_z         1.1704     0.8544    1.1405    1.2011
I(dirPopularity2_z^2)    0.9888     1.0114    0.9857    0.9918

Concordance= 0.557  (se = 0.01 )
Rsquare= 0.002   (max possible= 0.713 )
Likelihood ratio test= 172.1  on 2 df,   p=0
Wald test            = 188.8  on 2 df,   p=0
Score (logrank) test = 191.2  on 2 df,   p=0


In [67]:
summary(m03)

Call:
coxph(formula = Surv(rep(1, nrow(dtrem2)), eventDummy) ~ fourCycle2_z + 
    strata(eventTime), data = dtrem2)

  n= 93325, number of events= 7374 

                 coef exp(coef) se(coef)    z Pr(>|z|)    
fourCycle2_z 0.036312  1.036979 0.004364 8.32   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

             exp(coef) exp(-coef) lower .95 upper .95
fourCycle2_z     1.037     0.9643     1.028     1.046

Concordance= 0.51  (se = 0.003 )
Rsquare= 0.001   (max possible= 0.713 )
Likelihood ratio test= 54.88  on 1 df,   p=1.279e-13
Wald test            = 69.22  on 1 df,   p=1.11e-16
Score (logrank) test = 71.45  on 1 df,   p=0


In [68]:
summary(m04)

Call:
coxph(formula = Surv(rep(1, nrow(dtrem2)), eventDummy) ~ dirPopularity2_z + 
    I(dirPopularity2_z^2) + fourCycle2_z + strata(eventTime), 
    data = dtrem2)

  n= 93325, number of events= 7374 

                           coef exp(coef)  se(coef)      z Pr(>|z|)    
dirPopularity2_z       0.147815  1.159299  0.013489 10.958  < 2e-16 ***
I(dirPopularity2_z^2) -0.011034  0.989027  0.001569 -7.030 2.06e-12 ***
fourCycle2_z           0.021158  1.021383  0.004653  4.547 5.44e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

                      exp(coef) exp(-coef) lower .95 upper .95
dirPopularity2_z          1.159     0.8626     1.129    1.1904
I(dirPopularity2_z^2)     0.989     1.0111     0.986    0.9921
fourCycle2_z              1.021     0.9791     1.012    1.0307

Concordance= 0.558  (se = 0.01 )
Rsquare= 0.002   (max possible= 0.713 )
Likelihood ratio test= 190.6  on 3 df,   p=0
Wald test            = 214.5  on 3 df,   p=0
Score (logrank) test = 21

In [70]:
options(scipen = 999)
summary(m05)

Call:
coxph(formula = Surv(rep(1, nrow(dtrem2)), eventDummy) ~ dirPopularity2_z + 
    I(dirPopularity2_z^2) + fourCycle2_z + gender_man + nationality_dk + 
    entity_finb + companyOperatingRevenue + strata(eventTime), 
    data = dtrem2)

  n= 55901, number of events= 4596 
   (37424 observations deleted due to missingness)

                                   coef       exp(coef)        se(coef)      z
dirPopularity2_z         0.120783029444  1.128380060557  0.014976509504  8.065
I(dirPopularity2_z^2)   -0.008591678831  0.991445124166  0.001617501832 -5.312
fourCycle2_z             0.024456795004  1.024758315472  0.005720451407  4.275
gender_man               0.116280214282  1.123310595708  0.053096239675  2.190
nationality_dk           0.117894748578  1.125125684057  0.052723033923  2.236
entity_finb              0.072777132311  1.075490818050  0.046773958696  1.556
companyOperatingRevenue -0.000000001459  0.999999998541  0.000000005489 -0.266
                                    Pr(

### Model loglikelihood

In [73]:
anova(m01,m02,m03,m04)

loglik,Chisq,Df,P(>|Chi|)
-58246.20,NA,NA,NA
-58214.30,63.80073,1,0.000000000000001332268
-58272.93,117.25183,1,0.000000000000000000000
-58205.09,135.67006,2,0.000000000000000000000


In [77]:
anova(m05)

,loglik,Chisq,Df,Pr(>|Chi|)
NULL,-34164.94,NA,NA,NA
dirPopularity2_z,-34132.94,64.01656116,1,0.000000000000001221245
I(dirPopularity2_z^2),-34113.77,38.32578544,1,0.000000000598663452145
fourCycle2_z,-34105.66,16.21648219,1,0.000056500393034841778
gender_man,-34102.69,5.95425921,1,0.014681770876384336866
nationality_dk,-34099.86,5.65717293,1,0.017384122286483671971
entity_finb,-34098.68,2.34735018,1,0.125496167710754558122
companyOperatingRevenue,-34098.65,0.07153485,1,0.789115010634509506104


In [79]:
m01$loglik; 
m02$loglik; 
m03$loglik;
m04$loglik;
m05$loglik;

[1] -58300.37 -58246.20

[1] -58300.37 -58214.30

[1] -58300.37 -58272.93

[1] -58300.37 -58205.09

[1] -34164.94 -34098.65